# NLI base results: Trendyol-LLM-7b-chat-v2.0 (Trendyol/Trendyol-LLM-7b-chat-v2.0) on Colab GPU

Loads [yilmazzey/sdp2-nli](https://huggingface.co/datasets/yilmazzey/sdp2-nli) (snli_tr_1_1, multinli_tr_1_1, trglue_mnli) and runs **test-only** zero-shot Turkish NLI evaluation with **Trendyol-LLM-7b-chat-v2.0** on Colab GPU.

Uses 4-bit quantization (bitsandbytes NF4 + double quant) to fit on T4 16GB. Llama-3 / Mistral-style chat format with a strong Turkish+English system prompt forcing a single-word answer. Outputs parsed to 0=entailment, 1=neutral, 2=contradiction.

**Splits:** snli → test; multinli → validation_matched/mismatched; trglue → test_matched/test_mismatched. **Metrics:** Accuracy, macro F1, per-class F1, confusion matrix (CSV + seaborn plot). Results saved to `./results/`.

In [1]:
# Colab: run once (Runtime -> Change runtime type -> GPU)
!pip install -q -U transformers datasets accelerate scikit-learn tqdm bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.7/520.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 142.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 54.5 MB/s eta 0:00:00


In [2]:
import json
import random
import re
from collections import Counter
from pathlib import Path

import numpy as np
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm
from transformers import AutoTokenizer, BitsAndBytesConfig, pipeline

try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_PLOT = True
except ImportError:
    HAS_PLOT = False

LABEL_NAMES = ["entailment", "neutral", "contradiction"]

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU; 4-bit loading may still run on CPU but will be slow.")

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

GPU: NVIDIA A100-SXM4-40GB


In [9]:
REPO_ID = "yilmazzey/sdp2-nli"
CONFIGS = ["snli_tr_1_1", "multinli_tr_1_1", "trglue_mnli"]
MODEL_ID = "Trendyol/Trendyol-LLM-7b-chat-v1.0"
NUM_LABELS = 3
RESULTS_DIR = "results"
BATCH_SIZE = 16  # Safe and fast on Colab T4 with 4-bit
EVAL_SPLITS = {
    "snli_tr_1_1": ["test"],
    "multinli_tr_1_1": ["validation_matched", "validation_mismatched"],
    "trglue_mnli": ["test_matched", "test_mismatched"],
}

In [4]:
datasets = {}
for cfg in CONFIGS:
    print(f"Loading {REPO_ID} :: {cfg} ...")
    datasets[cfg] = load_dataset(REPO_ID, cfg)
    print("  splits:", list(datasets[cfg].keys()))

Loading yilmazzey/sdp2-nli :: snli_tr_1_1 ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

snli_tr_1_1/train-00000-of-00001.parquet:   0%|          | 0.00/25.4M [00:00<?, ?B/s]

snli_tr_1_1/validation-00000-of-00001.pa(…):   0%|          | 0.00/558k [00:00<?, ?B/s]

...

Generating validation_matched split:   0%|          | 0/9050 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9200 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9217 [00:00<?, ? examples/s]

  splits: ['train', 'validation_matched', 'validation_mismatched', 'test_matched', 'test_mismatched']


In [7]:
from huggingface_hub import login

login()  # use token from huggingface-cli login or HF_TOKEN env

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

MODEL_ID = "Trendyol/Trendyol-LLM-7b-chat-v1.0"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False, trust_remote_code=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading model in 4-bit...")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,          # only here
    torch_dtype=torch.bfloat16,
)

print("Creating pipeline...")
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=None,                 # silence max_length warning
)

print("Model loaded successfully.")

Loading tokenizer...
Loading model in 4-bit...


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'max_length'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


Creating pipeline...
Model loaded successfully.


In [15]:
# Llama/Mistral-style: [INST] <<SYS>> system <</SYS>> user_content [/INST]
# System prompt: Turkish + English, force single word
SYSTEM_PROMPT = "Sadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma."


def nli_prompt(premise, hypothesis):
    user_part = (
        f"Premise: {premise}\n"
        f"Hypothesis: {hypothesis}\n"
        "Does the premise entail, is neutral to, or contradict the hypothesis? "
        "Answer with only one word: entailment, neutral, or contradiction."
    )
    return f"[INST] <<SYS>>\n{SYSTEM_PROMPT}\n<</SYS>>\n\n{user_part} [/INST]"


LABEL_WORD_TO_ID = {
    "entailment": 0,
    "neutral": 1,
    "contradiction": 2,
    "içerme": 0,
    "tarafsız": 1,
    "nötr": 1,
    "çelişki": 2,
}


def parse_generated_label(generated_text: str, prompt_text: str) -> int:
    """First word after prompt; lowercase; strip punctuation; map EN+TR; default 1 (neutral)."""
    continuation = generated_text
    if generated_text.startswith(prompt_text):
        continuation = generated_text[len(prompt_text):]
    continuation = continuation.strip()
    if not continuation:
        return 1
    first = continuation.split()[0] if continuation.split() else ""
    first = re.sub(r"[.,;:!?\"'()\[\]]+", "", first).strip().lower()
    if not first:
        return 1
    return LABEL_WORD_TO_ID.get(first, 1)


def run_prompted_inference(ds):
    premises = ds["premise"]
    hypotheses = ds["hypothesis"]
    labels = ds["label"]
    n = len(labels)
    prompts = [nli_prompt(p, h) for p, h in zip(premises, hypotheses)]
    y_pred = []
    debug_indices = set(list(range(min(5, n))) + list(range(0, n, 100)))

    for start in tqdm(range(0, n, BATCH_SIZE), desc="Inference"):
        batch_prompts = prompts[start : start + BATCH_SIZE]
        out = generator(
            batch_prompts,
            max_new_tokens=5,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        )
        for i, prompt_text in enumerate(batch_prompts):
            gen_text = out[i][0]["generated_text"]
            label_id = parse_generated_label(gen_text, prompt_text)
            y_pred.append(label_id)
            idx = start + i
            if idx in debug_indices:
                print(f"[sample {idx}] raw: {repr(gen_text)} -> parsed: {label_id} ({LABEL_NAMES[label_id]})")

    y_true = np.array(labels, dtype=np.int64)
    y_pred = np.array(y_pred, dtype=np.int64)
    print("True label dist:", dict(Counter(y_true)))
    print("Pred label dist:", dict(Counter(y_pred)))
    return y_true, y_pred

In [16]:
def compute_metrics(y_true, y_pred):
    acc = float(accuracy_score(y_true, y_pred))
    f1_macro = float(f1_score(y_true, y_pred, average="macro", zero_division=0))
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = {LABEL_NAMES[i]: float(f1_per_class[i]) for i in range(NUM_LABELS)}
    cm = confusion_matrix(y_true, y_pred)
    out = {"accuracy": acc, "f1_macro": f1_macro, "f1_per_class": f1_per_class}
    return out, cm


def save_confusion_plot(cm, path):
    if not HAS_PLOT:
        return
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES, ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

In [17]:
Path(RESULTS_DIR).mkdir(parents=True, exist_ok=True)
all_metrics = {}

for config_name in CONFIGS:
    ds_dict = datasets[config_name]
    split_names = EVAL_SPLITS[config_name]
    all_metrics[config_name] = {}

    for split_name in split_names:
        if split_name not in ds_dict:
            print(f"  Skip {config_name}/{split_name} (missing)")
            continue
        ds = ds_dict[split_name]
        print(f"Evaluating {config_name} / {split_name} ...")
        y_true, y_pred = run_prompted_inference(ds)
        metrics, cm = compute_metrics(y_true, y_pred)
        all_metrics[config_name][split_name] = metrics

        cm_path = Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.csv"
        np.savetxt(cm_path, cm, fmt="%d", delimiter=",")
        save_confusion_plot(cm, Path(RESULTS_DIR) / f"confusion_{config_name}_{split_name}.png")

        print(f"  accuracy={metrics['accuracy']:.4f}, f1_macro={metrics['f1_macro']:.4f}")

with open(Path(RESULTS_DIR) / "metrics.json", "w") as f:
    json.dump(all_metrics, f, indent=2)
print(f"Saved {RESULTS_DIR}/metrics.json")

Evaluating snli_tr_1_1 / test ...



Inference:   0%|          | 0/614 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Passing `generation_config` together with generation-related arguments=({'pad_token_id', 'temperature', 'do_sample', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.

Inference:   0%|          | 1/614 [00:08<1:22:52,  8.11s/it]

[sample 0] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma.\n<</SYS>>\n\nPremise: Bu kilise korosu, kilisedeki kitaptan neşeli şarkılar söylerken kitlelere şarkı söyler.\nHypothesis: Kilisenin tavanında çatlaklar var.\nDoes the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction. [/INST] ▁ ▁Ne ut ral.' -> parsed: 1 (neutral)
[sample 1] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma.\n<</SYS>>\n\nPremise: Bu kilise korosu, kilisedeki kitaptan neşeli şarkılar söylerken kitlelere şarkı söyler.\nHypothesis: Kilise şarkıyla dolu.\nDoes the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction. [/INST] ▁ ▁Ne ut ral.' -> parsed: 1 (neutral)
[sample 2] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral


Inference:   1%|          | 7/614 [00:46<1:04:40,  6.39s/it]

[sample 100] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma.\n<</SYS>>\n\nPremise: Orkestradaki bir kadın keman çalıyor.\nHypothesis: Bir adam teleskopla bakıyor.\nDoes the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction. [/INST] ▁ ▁Ne ut ral.' -> parsed: 1 (neutral)


...

[sample 9100] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma.\n<</SYS>>\n\nPremise: Çiftlerden birinin iş hayatındaki olumlu ya da olumsuz değişiklik, akut veya kronik hastalıklar, ölüm gibi karşılaşılabilecek ve kişide stres yaratabilecek yaşamsal durumlar vardır.\nHypothesis: Çiftlerin hayatında stres yoktur.\nDoes the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction. [/INST] ▁ ▁Ne ut ral.' -> parsed: 1 (neutral)



Inference: 100%|█████████▉| 576/577 [1:00:30<00:06,  6.31s/it]

[sample 9200] raw: '[INST] <<SYS>>\nSadece tek kelime cevap ver: entailment, neutral veya contradiction. Hiçbir açıklama yazma.\n<</SYS>>\n\nPremise: Hassas cildiniz için nemlendirici ve diğer bakım ürünlerini seçerken alkol, paraben, parfüm ve asit içeren ürünlerden kaçınmalısınız.\nHypothesis: Günlük cilt bakımı için sabun gibi basit ürünler yeterli olabilir.\nDoes the premise entail, is neutral to, or contradict the hypothesis? Answer with only one word: entailment, neutral, or contradiction. [/INST] ▁ ▁Ne ut ral.' -> parsed: 1 (neutral)



Inference: 100%|██████████| 577/577 [1:00:30<00:00,  6.29s/it]


True label dist: {np.int64(1): 3043, np.int64(0): 3101, np.int64(2): 3073}
Pred label dist: {np.int64(1): 9217}
  accuracy=0.3302, f1_macro=0.1655
Saved results/metrics.json


In [18]:
# Summary: per config/split
for config_name, splits in all_metrics.items():
    for split_name, m in splits.items():
        print(f"{config_name} / {split_name}: acc={m['accuracy']:.4f}, F1_macro={m['f1_macro']:.4f}, F1_per_class={m['f1_per_class']}")

snli_tr_1_1 / test: acc=0.3277, F1_macro=0.1645, F1_per_class={'entailment': 0.0, 'neutral': 0.49359809859694853, 'contradiction': 0.0}
multinli_tr_1_1 / validation_matched: acc=0.3184, F1_macro=0.1610, F1_per_class={'entailment': 0.0, 'neutral': 0.4829879369007114, 'contradiction': 0.0}
multinli_tr_1_1 / validation_mismatched: acc=0.3185, F1_macro=0.1610, F1_per_class={'entailment': 0.0, 'neutral': 0.48309402501157944, 'contradiction': 0.0}
trglue_mnli / test_matched: acc=0.3484, F1_macro=0.1722, F1_per_class={'entailment': 0.0, 'neutral': 0.5167133212580274, 'contradiction': 0.0}
trglue_mnli / test_mismatched: acc=0.3302, F1_macro=0.1655, F1_per_class={'entailment': 0.0, 'neutral': 0.4964110929853181, 'contradiction': 0.0}
